In [1]:
import pandas as pd
import numpy as np
import joblib
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model

In [2]:
df = pd.read_csv('Data/model_ready.csv', index_col = 0)

In [3]:
xgb_model = joblib.load('Models/xgb_clf')
scaler = joblib.load('Models/scaler')

In [4]:
keras_model = load_model('Models/mlp')

In [5]:
df.columns

Index(['Winner', 'R_fighter', 'Fighter', 'Height', 'Reach', 'Stance', 'Weight',
       'age', 'loss', 'win',
       ...
       'Open Stance', 'Orthodox', 'Sideways', 'Southpaw', 'Switch',
       'Open Stance.1', 'Orthodox.1', 'Sideways.1', 'Southpaw.1', 'Switch.1'],
      dtype='object', length=124)

In [6]:
#Relevant cumulative statistics of fighter's most recent fight:
fighter_columns = ['Fighter', 'Height', 'Reach', 'Stance', 'Weight',
       'age', 'loss', 'win', 'draws', 'streak', 'cum_match_time', 'avg_cum_KD',
       'avg_cum_Sub. att', 'avg_cum_Pass', 'avg_cum_Rev.',
       'avg_cum_Sig. str. Hits', 'avg_cum_Sig. str. Attempts',
       'avg_cum_Total str. Hits', 'avg_cum_Total str. Attempts',
       'avg_cum_Td Hits', 'avg_cum_Td Attempts', 'avg_cum_Head Hits',
       'avg_cum_Head Attempts', 'avg_cum_Body Hits', 'avg_cum_Body Attempts',
       'avg_cum_Leg Hits', 'avg_cum_Leg Attempts', 'avg_cum_Distance Hits',
       'avg_cum_Distance Attempts', 'avg_cum_Clinch Hits',
       'avg_cum_Clinch Attempts', 'avg_cum_Ground Hits',
       'avg_cum_Ground Attempts', 'eff_sig_str', 'eff_hits', 'eff_tds',
       'eff_head', 'eff_leg', 'eff_body', 'eff_distance', 'eff_clinch',
       'eff_ground', 'matches', 'Open Stance', 'Orthodox', 'Sideways', 'Southpaw', 'Switch']

#Relevant cumulative statistics of fighter.1's most recent fight:
fighter_1_columns = ['Fighter.1', 'Height.1', 'Reach.1', 'Stance.1',
       'Weight.1', 'age.1', 'loss.1', 'win.1', 'draws.1', 'streak.1',
       'cum_match_time.1', 'avg_cum_KD.1', 'avg_cum_Sub. att.1',
       'avg_cum_Pass.1', 'avg_cum_Rev..1', 'avg_cum_Sig. str. Hits.1',
       'avg_cum_Sig. str. Attempts.1', 'avg_cum_Total str. Hits.1',
       'avg_cum_Total str. Attempts.1', 'avg_cum_Td Hits.1',
       'avg_cum_Td Attempts.1', 'avg_cum_Head Hits.1',
       'avg_cum_Head Attempts.1', 'avg_cum_Body Hits.1',
       'avg_cum_Body Attempts.1', 'avg_cum_Leg Hits.1',
       'avg_cum_Leg Attempts.1', 'avg_cum_Distance Hits.1',
       'avg_cum_Distance Attempts.1', 'avg_cum_Clinch Hits.1',
       'avg_cum_Clinch Attempts.1', 'avg_cum_Ground Hits.1',
       'avg_cum_Ground Attempts.1', 'eff_sig_str.1', 'eff_hits.1', 'eff_tds.1',
       'eff_head.1', 'eff_leg.1', 'eff_body.1', 'eff_distance.1',
       'eff_clinch.1', 'eff_ground.1', 'matches.1', 'Open Stance.1', 'Orthodox.1', 'Sideways.1', 'Southpaw.1', 'Switch.1']

#List of weight classes
weight_classes = ['Bantamweight', 'Catch Weight', 'Featherweight', 'Flyweight',
       'Heavyweight', 'Light Heavyweight', 'Lightweight', 'Middleweight',
       'Open Weight', 'Super Heavyweight', 'Welterweight',
       "Women's Bantamweight", "Women's Featherweight", "Women's Flyweight",
       "Women's Strawweight"]

In [7]:
#Helper function that determines if fighter is Fighter or Fighter.1 in a particular match
def zero_or_one_fighter(match, fighter):
    if match['Fighter'] == fighter:
        return 0
    else:
        return 1

#Helper function that adds the correct one-hot-encoded weight_class information:
def get_weight_class(wt_class):
    weight = pd.DataFrame([0]*len(weight_classes)).T
    weight.columns = weight_classes
    
    try:
        weight[wt_class] = 1
        return weight
    except:
        return weight

#Retrieve complete statistics of fighter's most recent fight
def get_recent(fighter):
    temp = df[df['Fighter'] == fighter]
    temp_1 = df[df['Fighter.1'] == fighter]
    
    combined = pd.concat([temp, temp_1], axis = 0).sort_index()
    return combined.iloc[0, :]


    
#Returns a details of a hypothetical match between fighter and fighter_1 using cumulative statistics
#of their most recent match:
def get_match(fighter, fighter_1):
    
    recent = get_recent(fighter)
    recent_1 = get_recent(fighter_1)
    
    if zero_or_one_fighter(recent, fighter) == 0:
        recent_stats = recent[fighter_columns]
    else:
        recent_stats = recent[fighter_1_columns]
        
    if zero_or_one_fighter(recent_1, fighter_1) == 0:
        recent_stats_1 = recent_1[fighter_columns]
    else:
        recent_stats_1 = recent_1[fighter_1_columns]
        
    combined = pd.DataFrame(pd.concat([recent_stats, recent_stats_1], axis = 0)).T
    combined.columns = fighter_columns+fighter_1_columns
    
    return combined

In [8]:
drop_columns = ['Fighter', 'Height', 'Reach', 'Stance', 'Weight', 'Fighter.1', 
                'Height.1', 'Reach.1', 'Stance.1', 'Weight.1']

In [9]:
def to_model_format(match, weight):
    match['Reach_diff'] = match['Reach'] - match['Reach.1']
    match['Height_diff'] = match['Height'] - match['Height.1']
    match['Weight_diff'] = match['Weight'] - match['Weight.1']

    match = pd.concat([match, get_weight_class(weight)], axis = 1)

    
    match.drop(drop_columns, inplace = True, axis = 1)

    return match


In [10]:
test_match = get_match('Charles Oliveira', 'Gilbert Burns')
to_model_format(test_match, '')

,age,loss,win,draws,streak,cum_match_time,avg_cum_KD,avg_cum_Sub. att,avg_cum_Pass,avg_cum_Rev.,...,Lightweight,Middleweight,Open Weight,Super Heavyweight,Welterweight,Women's Bantamweight,Women's Featherweight,Women's Flyweight,Women's Strawweight,
0,30.43,8,16,1,6,162.18,0.25,1.85,2.1,0.25,...,0,0,0,0,0,0,0,0,0,1


# New Match Pipeline Scrape

In [11]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime as dt

In [82]:
url = 'http://www.ufcstats.com/event-details/dfb965c9824425db'

In [83]:
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [84]:
#Removes multiple white spaces (for better implementation see gensim's preprocessing utils)
def remove_space_lines(text):
    pattern1 = re.compile(r'[\s\s+]')
    return re.sub(pattern1, ' ', text)

def get_fight_auxiliary(soup):
    '''
    Input: beautifulsoup of an event url: (ie. http://www.ufcstats.com/event-details/53278852bcd91e11)
    Outputs: pandas Series
        date, location, attendance
    '''
    
    table = []
    
    auxiliary_table = soup.find_all('li', {'class': 'b-list__box-list-item'})
    for item in auxiliary_table:
        attribute = remove_space_lines(item.text).strip()

        #If attribute is missing, replace with ''
        try:
            attribute = re.findall(r'\s\s+(.*)', attribute)[0]
        except:
            attribute = '' 
        
        table.append(attribute)
        
    table_series = pd.Series(table)
    table_series.index = ['date', 'location', 'attendance']
    
    if table_series['attendance'] != '':
        table_series['attendance'] = re.sub(',', '', table_series['attendance'])
        table_series['attendance'] = int(table_series['attendance'])
    
    table_series['date'] = dt.strptime(table_series['date'], '%B %d, %Y').strftime('%d-%m-%Y')

    return table_series

#Determine if observation is a title-bout
def find_belt(img_tag):
    try:
        image_link = img_tag['src']
        if re.match(r'.*belt.*', image_link) != None:
            return True
    except:
        return False

In [85]:
table_rows = soup.find_all('p', {'class': 'b-fight-details__table-text'})

try:

    name_class = []

    for i in table_rows:
        name_class.append(remove_space_lines(i.text).strip())

    name_class = [i for i in name_class if i != '']
    name_class = np.array(name_class).reshape(-1, 4)

    name_class = pd.DataFrame(name_class, columns = ['Fighter', 'Fighter.1', 'Drop', 'Weight_class'])
    name_class.drop('Drop', inplace = True, axis = 1)
    name_class
    
except:
    
    names = soup.find_all('a', {'class': 'b-link b-link_style_black'})
    name_list = [remove_space_lines(item.text).strip() for item in names]
    name_class = pd.DataFrame(np.array(name_list).reshape(-1, 2))

    wt_class = pd.read_html(soup.find_all('table')[0].prettify())[0]['Weight class']
    name_class['Weight_class'] = wt_class

    name_class.columns = ['Fighter', 'Fighter.1', 'Weight_class']

In [98]:
name_class

,Fighter,Fighter.1,Weight_class
0,Alistair Overeem,Walt Harris,Heavyweight
1,Claudia Gadelha,Angela Hill,Women's Strawweight
2,Dan Ige,Edson Barboza,Featherweight
3,Krzysztof Jotko,Eryk Anders,Middleweight
4,Song Yadong,Marlon Vera,Featherweight
5,Miguel Baeza,Matt Brown,Welterweight
6,Kevin Holland,Anthony Hernandez,Middleweight
7,Giga Chikadze,Irwin Rivera,Featherweight
8,Nate Landwehr,Darren Elkins,Featherweight
9,Cortney Casey,Mara Romero Borella,Women's Flyweight


# Predictions:

In [162]:
new_matches = pd.DataFrame()

fighters = []
fighters_1 = []

errors = []

for row in name_class.iterrows():
    entry = row[1]
    try:
        print('Fight:', entry['Fighter'], 'v', entry['Fighter.1'])
        match = get_match(entry['Fighter'], entry['Fighter.1'])    
        final = to_model_format(match, entry['Weight_class'])
        fighters.append(entry['Fighter'])
        fighters_1.append(entry['Fighter.1'])
        new_matches = new_matches.append(final)
    except:
        print('error', entry['Fighter.1'], entry['Fighter'])
        errors.append(entry['Fighter'])
        errors.append(entry['Fighter.1'])
        pass

Fight: Alistair Overeem v Walt Harris
Fight: Claudia Gadelha v Angela Hill
Fight: Dan Ige v Edson Barboza
Fight: Krzysztof Jotko v Eryk Anders
Fight: Song Yadong v Marlon Vera
Fight: Miguel Baeza v Matt Brown
Fight: Kevin Holland v Anthony Hernandez
Fight: Giga Chikadze v Irwin Rivera
error Irwin Rivera Giga Chikadze
Fight: Nate Landwehr v Darren Elkins
Fight: Cortney Casey v Mara Romero Borella
Fight: Rodrigo Nascimento v Don'Tale Mayes
error Don'Tale Mayes Rodrigo Nascimento


### Debug & Add New Fighters to Database:

In [175]:
for i in errors:
    try:
        get_recent(i)
    except:
        print(i)
        pass

Irwin Rivera
Rodrigo Nascimento


In [218]:
player_url = ['http://www.ufcstats.com/fighter-details/06a80a685ea98573', 
              'http://www.ufcstats.com/fighter-details/d31a5546c0e9b213']

In [212]:
def number_finder(text):
    pattern = re.compile(r'\d+')
    return re.search(pattern, text)[0]
    
def convert_to_inches(feet, inches):
    return 12*feet + inches
        
def get_height(text):
    
    height = []
    ft_in = text.split("'")
    
    for item in ft_in:
        try:
            height.append(int(number_finder(item)[0]))
        except:
            height.append(99999)
    
    return convert_to_inches(height[0], height[1])

def get_dob(text):
    text = text.strip()
    return dt.strptime(text, '%b %d, %Y').strftime('%d-%m-%Y')
    


def get_complete_df(attr_list):
    
    data = []
    try:
        data.append(get_height(attr_list[0][1])) #height in inches
    except: 
        data.append(0)
        
    try:
        data.append(int(number_finder(attr_list[1][1]))) #weight in pounds
    except:
        data.append(0)
    
    try:
        data.append(int(number_finder(attr_list[2][1]))) #reach in inches
    except:
        data.append(0)
        
    data.append(attr_list[3][1]) #Stance
    
    try:
        data.append(get_dob(attr_list[4][1])) #Get DOB
    except:    
        data.append(0)
    
    complete_df = pd.Series(data, index = ['Height', 'Weight', 'Reach', 'Stance', 'DOB'])
    return complete_df    
    

In [213]:
def get_name(soup):
    return remove_space_lines(soup.find('span', {'class': 'b-content__title-highlight'}).text).strip()

def get_player_attr(soup):
    
    name = get_name(soup)
    
    player_attr = soup.find('ul', {'class': 'b-list__box-list'})
    player_attr = player_attr.find_all('li')

    player_attr_list = []
    
    for item in player_attr:
        player_attr_list.append(remove_space_lines(item.text).strip())

    player_attr_list = list(map(lambda x: x.split(':'), player_attr_list))  
    #print(player_attr_list)    
    
    
    complete_df = get_complete_df(player_attr_list)
    complete_df['Name'] = name
    
    return complete_df
    

In [220]:
new_fighters = pd.DataFrame()

for url in player_url:
    player_page = requests.get(url)
    soup = BeautifulSoup(player_page.content, 'html.parser')
    
    new_fighters = new_fighters.append(pd.DataFrame(get_player_attr(soup)).T)
    

In [222]:
fighters_data = pd.read_csv('Data/fighter.csv', index_col = 0)
#fighters_data

In [223]:
fighters_data = fighters_data.append(new_fighters)

In [224]:
fighters_data

,DOB,Height,Name,Reach,Stance,Weight
0,26-08-1994,66,Gabriel Silva,71,Orthodox,135
0,20-09-1989,72,Aalon Cruz,78,Switch,145
0,05-11-1986,66,Davi Ramos,70,Orthodox,155
0,17-07-1976,79,Sean McCorkle,81,Orthodox,265
0,28-02-1991,72,Glaico Franca,77,NaN,155
...,...,...,...,...,...,...
0,20-06-1978,73,Rampage Jackson,73,Orthodox,205
0,06-01-1983,61,Johnny Bedford,71,Orthodox,135
0,14-04-1977,67,Jessica Rakoczy,68,Orthodox,115
0,14-02-1989,66,Irwin Rivera,0,Southpaw,145


In [228]:
fighters_data.to_csv('Data/fighter.csv')

Fighters do not have previous match experience, no prediction available

### Continue:

In [153]:
new_matches = new_matches.astype(float)
new_matches['title_bout'] = 0
new_matches['num_rounds'] = 3

orig_columns = ['age', 'loss', 'win', 'draws', 'streak', 'cum_match_time', 'avg_cum_KD', 'avg_cum_Sub. att', 
                'avg_cum_Pass', 'avg_cum_Rev.', 'avg_cum_Sig. str. Hits', 'avg_cum_Sig. str. Attempts', 
                'avg_cum_Total str. Hits', 'avg_cum_Total str. Attempts', 'avg_cum_Td Hits', 'avg_cum_Td Attempts', 
                'avg_cum_Head Hits', 'avg_cum_Head Attempts', 'avg_cum_Body Hits', 'avg_cum_Body Attempts', 
                'avg_cum_Leg Hits', 'avg_cum_Leg Attempts', 'avg_cum_Distance Hits', 'avg_cum_Distance Attempts', 
                'avg_cum_Clinch Hits', 'avg_cum_Clinch Attempts', 'avg_cum_Ground Hits', 'avg_cum_Ground Attempts', 
                'eff_sig_str', 'eff_hits', 'eff_tds', 'eff_head', 'eff_leg', 'eff_body', 'eff_distance', 'eff_clinch', 
                'eff_ground', 'age.1', 'loss.1', 'win.1', 'draws.1', 'streak.1', 'cum_match_time.1', 'avg_cum_KD.1', 
                'avg_cum_Sub. att.1', 'avg_cum_Pass.1', 'avg_cum_Rev..1', 'avg_cum_Sig. str. Hits.1', 
                'avg_cum_Sig. str. Attempts.1', 'avg_cum_Total str. Hits.1', 'avg_cum_Total str. Attempts.1', 
                'avg_cum_Td Hits.1', 'avg_cum_Td Attempts.1', 'avg_cum_Head Hits.1', 'avg_cum_Head Attempts.1', 
                'avg_cum_Body Hits.1', 'avg_cum_Body Attempts.1', 'avg_cum_Leg Hits.1', 'avg_cum_Leg Attempts.1', 
                'avg_cum_Distance Hits.1', 'avg_cum_Distance Attempts.1', 'avg_cum_Clinch Hits.1', 'avg_cum_Clinch Attempts.1',
                'avg_cum_Ground Hits.1', 'avg_cum_Ground Attempts.1', 'eff_sig_str.1', 'eff_hits.1', 'eff_tds.1', 'eff_head.1', 
                'eff_leg.1', 'eff_body.1', 'eff_distance.1', 'eff_clinch.1', 'eff_ground.1', 'title_bout', 'num_rounds', 
                'matches', 'matches.1', 'Reach_diff', 'Weight_diff', 'Height_diff', 'Bantamweight', 'Catch Weight', 
                'Featherweight', 'Flyweight', 'Heavyweight', 'Light Heavyweight', 'Lightweight', 'Middleweight', 'Open Weight', 
                'Super Heavyweight', 'Welterweight', "Women's Bantamweight", "Women's Featherweight", "Women's Flyweight", 
                "Women's Strawweight", 'Open Stance', 'Orthodox', 'Sideways', 'Southpaw', 'Switch', 'Open Stance.1', 
                'Orthodox.1', 'Sideways.1', 'Southpaw.1', 'Switch.1']

scale_columns = ['age','loss','win','draws','streak','cum_match_time','avg_cum_KD',
                 'avg_cum_Sub. att', 'avg_cum_Pass','avg_cum_Rev.','avg_cum_Sig. str. Hits','avg_cum_Sig. str. Attempts',
                 'avg_cum_Total str. Hits','avg_cum_Total str. Attempts','avg_cum_Td Hits','avg_cum_Td Attempts',
                 'avg_cum_Head Hits','avg_cum_Head Attempts','avg_cum_Body Hits','avg_cum_Body Attempts',
                 'avg_cum_Leg Hits','avg_cum_Leg Attempts','avg_cum_Distance Hits','avg_cum_Distance Attempts',
                 'avg_cum_Clinch Hits','avg_cum_Clinch Attempts','avg_cum_Ground Hits','avg_cum_Ground Attempts',
                 'eff_sig_str','eff_hits','eff_tds','eff_head','eff_leg','eff_body','eff_distance','eff_clinch','eff_ground',
                 'age.1','loss.1','win.1','draws.1','streak.1','cum_match_time.1',
                 'avg_cum_KD.1','avg_cum_Sub. att.1','avg_cum_Pass.1','avg_cum_Rev..1','avg_cum_Sig. str. Hits.1',
                 'avg_cum_Sig. str. Attempts.1','avg_cum_Total str. Hits.1','avg_cum_Total str. Attempts.1',
                 'avg_cum_Td Hits.1','avg_cum_Td Attempts.1','avg_cum_Head Hits.1','avg_cum_Head Attempts.1',
                 'avg_cum_Body Hits.1','avg_cum_Body Attempts.1','avg_cum_Leg Hits.1','avg_cum_Leg Attempts.1',
                 'avg_cum_Distance Hits.1','avg_cum_Distance Attempts.1','avg_cum_Clinch Hits.1','avg_cum_Clinch Attempts.1',
                 'avg_cum_Ground Hits.1','avg_cum_Ground Attempts.1','eff_sig_str.1','eff_hits.1','eff_tds.1','eff_head.1',
                 'eff_leg.1','eff_body.1','eff_distance.1','eff_clinch.1','eff_ground.1','num_rounds','matches',
                 'matches.1','Reach_diff','Weight_diff','Height_diff'
]

In [154]:
new_matches = new_matches[orig_columns]
new_matches

,age,loss,win,draws,streak,cum_match_time,avg_cum_KD,avg_cum_Sub. att,avg_cum_Pass,avg_cum_Rev.,...,Open Stance,Orthodox,Sideways,Southpaw,Switch,Open Stance.1,Orthodox.1,Sideways.1,Southpaw.1,Switch.1
0,39.58,6.0,10.0,0.0,2.0,124.35,0.72,0.24,0.24,0.00,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
0,30.60,4.0,5.0,0.0,-1.0,133.05,0.08,0.60,2.10,0.00,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
0,28.53,1.0,4.0,0.0,4.0,47.11,0.42,0.21,2.76,0.21,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
0,29.96,4.0,7.0,0.0,1.0,135.65,0.15,0.07,1.62,0.22,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
0,21.61,0.0,3.0,0.0,3.0,29.25,0.68,0.68,1.71,0.00,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
0,27.15,0.0,0.0,0.0,0.0,7.53,1.33,0.00,0.00,0.00,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
0,26.97,1.0,3.0,0.0,3.0,59.08,0.00,0.85,2.37,0.34,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
0,31.66,0.0,0.0,0.0,0.0,2.72,0.00,0.00,0.00,0.00,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
0,31.81,5.0,4.0,0.0,1.0,112.17,0.00,0.36,0.53,0.18,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [155]:
scaled = scaler.transform(new_matches[scale_columns])
xgb_predictions = pd.Series(xgb_model.predict_proba(new_matches)[:, 1])
nn_predictions = keras_model.predict(scaled)

In [156]:
def get_winner(names, predictions):
    
    winner = []
    loser = []
    
    for f0, f1, predict in zip(fighters, fighters_1, predictions):
        if predict >= .5:
            winner.append(f1)
            loser.append(f0)
        else:
            winner.append(f0)
            loser.append(f1)
            
    display = pd.DataFrame([winner, loser]).T
    display.columns = ['Winner', 'Loser']
            
    return display
    
    

In [157]:
xgb_predictions

0    0.429391
1    0.439572
2    0.258601
3    0.414023
4    0.549531
5    0.165606
6    0.188667
7    0.762392
8    0.462641
dtype: float32

In [158]:
nn_predictions

array([[0.41515958],
       [0.338266  ],
       [0.44494194],
       [0.39943942],
       [0.54173934],
       [0.40739164],
       [0.18918465],
       [0.81353056],
       [0.55342966]], dtype=float32)

In [159]:
get_winner(name_class, nn_predictions)

,Winner,Loser
0,Alistair Overeem,Walt Harris
1,Claudia Gadelha,Angela Hill
2,Dan Ige,Edson Barboza
3,Krzysztof Jotko,Eryk Anders
4,Marlon Vera,Song Yadong
5,Miguel Baeza,Matt Brown
6,Kevin Holland,Anthony Hernandez
7,Darren Elkins,Nate Landwehr
8,Mara Romero Borella,Cortney Casey


In [160]:
get_winner(name_class, xgb_predictions)

,Winner,Loser
0,Alistair Overeem,Walt Harris
1,Claudia Gadelha,Angela Hill
2,Dan Ige,Edson Barboza
3,Krzysztof Jotko,Eryk Anders
4,Marlon Vera,Song Yadong
5,Miguel Baeza,Matt Brown
6,Kevin Holland,Anthony Hernandez
7,Darren Elkins,Nate Landwehr
8,Cortney Casey,Mara Romero Borella
